## Solução para a Construção do Modelo Preditivo: Academia RedFit

- Enunciado: A academia RedFit deseja conhecer melhor o perfil de seus clientes para direcionar campanhas de marketing de forma mais eficaz. O objetivo é desenvolver um modelo de Machine Learning capaz de classificar os usuários em diferentes níveis de atividade física.

- **Objetivo**: Criar um modelo capaz de classificar **o perfil dos seus clientes** para direcionar a campanhas de marketing de forma mais eficaz.

### 1. Importação das Bibliotecas

In [149]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

from imblearn.over_sampling import SMOTE

### 2. Importar Base de Dados

In [150]:
df = pd.read_csv("data/academia_redfit.csv")

In [137]:
df

,idade,sexo,frequencia_semanal_treino,tipo_atividade,tempo_medio_exercicio,minutos_totais_semana,preco_plano,possui_nutricionista,primeiro_PGC,ultimo_PGC,data_matricula,estado
0,48,Outro,7.0,Natação,45.3,317.1,188.47,Não,39.5,24.9,2021-07-28,Ativa
1,30,Outro,7.0,Yoga,98.1,686.7,198.07,Não,25.2,23.5,2020-12-28,Ativa
2,35,Masculino,5.0,Cardio,33.9,169.5,154.90,Não,27.2,11.4,2022-02-27,Ativa
3,43,Outro,3.0,Yoga,75.6,226.8,250.79,Sim,14.4,19.2,2022-12-01,Ativa
4,37,Masculino,0.0,CrossFit,114.0,0.0,168.02,Sim,21.5,18.1,2024-01-20,Sedentária
...,...,...,...,...,...,...,...,...,...,...,...,...
995,43,Feminino,0.0,Futebol,101.7,0.0,215.21,Não,31.1,32.4,2024-04-24,Sedentária
996,39,Outro,5.0,Futebol,78.0,390.0,269.34,Sim,38.8,31.8,2022-11-02,Ativa
997,25,Feminino,2.0,Natação,112.3,224.6,130.93,Não,23.8,28.4,2025-01-24,Ativa
998,24,Masculino,4.0,CrossFit,113.8,455.2,149.22,Sim,30.5,35.7,2020-12-03,Ativa


### 3. Análise de Dados

In [152]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   idade                      1000 non-null   int64  
 1   sexo                       1000 non-null   object 
 2   frequencia_semanal_treino  949 non-null    float64
 3   tipo_atividade             1000 non-null   object 
 4   tempo_medio_exercicio      954 non-null    float64
 5   minutos_totais_semana      905 non-null    float64
 6   preco_plano                1000 non-null   float64
 7   possui_nutricionista       1000 non-null   object 
 8   primeiro_PGC               1000 non-null   float64
 9   ultimo_PGC                 1000 non-null   float64
 10  data_matricula             1000 non-null   object 
 11  estado                     1000 non-null   object 
dtypes: float64(6), int64(1), object(5)
memory usage: 93.9+ KB


In [153]:
df.groupby("tipo_atividade").size()

tipo_atividade
Cardio        181
CrossFit      164
Fut             9
Futebol       143
Musculação    160
Natacao         5
Natação       133
Soccer          4
Swimming        6
Yoga          195
dtype: int64

In [154]:
df.groupby("estado").size()

estado
Ativa         834
Sedentária    166
dtype: int64

### 4. Tratamento de Dados:

In [155]:
df["frequencia_semanal_treino"] = df["frequencia_semanal_treino"].fillna(df["frequencia_semanal_treino"].mean())
df["tempo_medio_exercicio"] = df["tempo_medio_exercicio"].fillna(df["tempo_medio_exercicio"].mean())
df["minutos_totais_semana"] = df["minutos_totais_semana"].fillna(df["minutos_totais_semana"].mean())


In [156]:
df.loc[df["tipo_atividade"] == "Fut", "tipo_atividade"] = "Futebol"
df.loc[df["tipo_atividade"] == "Soccer", "tipo_atividade"] = "Futebol"
df.loc[df["tipo_atividade"] == "Natacao", "tipo_atividade"] = "Natação"

In [143]:
df.groupby("tipo_atividade").size()

tipo_atividade
Cardio        181
CrossFit      164
Futebol       156
Musculação    160
Natação       138
Swimming        6
Yoga          195
dtype: int64

In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   idade                      1000 non-null   int64  
 1   sexo                       1000 non-null   object 
 2   frequencia_semanal_treino  1000 non-null   float64
 3   tipo_atividade             1000 non-null   object 
 4   tempo_medio_exercicio      1000 non-null   float64
 5   minutos_totais_semana      1000 non-null   float64
 6   preco_plano                1000 non-null   float64
 7   possui_nutricionista       1000 non-null   object 
 8   primeiro_PGC               1000 non-null   float64
 9   ultimo_PGC                 1000 non-null   float64
 10  data_matricula             1000 non-null   object 
 11  estado                     1000 non-null   object 
dtypes: float64(6), int64(1), object(5)
memory usage: 93.9+ KB


### 5. Separar a Base de Dados entre Features e Classes:

In [157]:
x_data = df.iloc[:, 0:11].values
y_data = df.iloc[:, 11].values

x_data, y_data

(array([[48, 'Outro', 7.0, ..., 39.5, 24.9, '2021-07-28'],
        [30, 'Outro', 7.0, ..., 25.2, 23.5, '2020-12-28'],
        [35, 'Masculino', 5.0, ..., 27.2, 11.4, '2022-02-27'],
        ...,
        [25, 'Feminino', 2.0, ..., 23.8, 28.4, '2025-01-24'],
        [24, 'Masculino', 4.0, ..., 30.5, 35.7, '2020-12-03'],
        [54, 'Masculino', 7.0, ..., 14.1, 25.8, '2025-09-08']],
       shape=(1000, 11), dtype=object),
 array(['Ativa', 'Ativa', 'Ativa', 'Ativa', 'Sedentária', 'Ativa', 'Ativa',
        'Ativa', 'Ativa', 'Ativa', 'Sedentária', 'Sedentária', 'Ativa',
        'Ativa', 'Ativa', 'Ativa', 'Ativa', 'Sedentária', 'Ativa', 'Ativa',
        'Ativa', 'Ativa', 'Ativa', 'Ativa', 'Ativa', 'Ativa', 'Sedentária',
        'Ativa', 'Ativa', 'Ativa', 'Ativa', 'Ativa', 'Ativa', 'Sedentária',
        'Ativa', 'Ativa', 'Sedentária', 'Ativa', 'Ativa', 'Ativa', 'Ativa',
        'Ativa', 'Ativa', 'Sedentária', 'Sedentária', 'Ativa', 'Ativa',
        'Ativa', 'Ativa', 'Ativa', 'Ativa', 'Ativa', 

#### 5.1 Segmentar colunas númericas e objetos

In [158]:
colunas_numericas = [index for index in range(x_data.shape[1]) if type(x_data[0][index]) == int or type(x_data[0][index]) == float]
colunas_objetos = [index for index in range(x_data.shape[1]) if type(x_data[0][index]) ==  str or type(x_data[0][index]) == np.str_] 

colunas_numericas, colunas_objetos

([0, 2, 4, 5, 6, 8, 9], [1, 3, 7, 10])

### 6. Aplicando Pré-processamento

In [159]:
encoder_cat = ColumnTransformer(transformers=[('OneHot', OrdinalEncoder(), colunas_objetos)], remainder='passthrough')

scaler = ColumnTransformer(transformers=[('Scaler', StandardScaler(), colunas_numericas)], remainder='passthrough')

preprocessor = Pipeline(steps=[('encoder', encoder_cat),
                                ('scaler', scaler)])

x_data = preprocessor.fit_transform(x_data)

x_data

array([[1.2042299952755242, -0.9723812481885865, 0.5328002989467715, ...,
        223.0, 317.1, 24.9],
       [1.2042299952755242, -0.9723812481885865, -0.8012385633486641,
        ..., 138.0, 686.7, 23.5],
       [-0.035965674421205046, -0.9723812481885865, -0.4306722127110431,
        ..., 320.0, 169.5, 11.4],
       ...,
       [-1.2761613441179345, -0.9723812481885865, -1.1718049139862852,
        ..., 714.0, 224.6, 28.4],
       [-0.035965674421205046, 1.0284032131048015, -1.2459181841138092,
        ..., 130.0, 455.2, 35.7],
       [-0.035965674421205046, 1.0284032131048015, 0.9774799197119166,
        ..., 805.0, 346.5, 25.8]], shape=(1000, 11), dtype=object)

### 7. Tratando Dados Desbalanceados

In [161]:
over_sampling = SMOTE(sampling_strategy="minority")
x_data, y_data = over_sampling.fit_resample(x_data, y_data)

x_data, y_data 

(array([[ 1.20423000e+00, -9.72381248e-01,  5.32800299e-01, ...,
          2.23000000e+02,  3.17100000e+02,  2.49000000e+01],
        [ 1.20423000e+00, -9.72381248e-01, -8.01238563e-01, ...,
          1.38000000e+02,  6.86700000e+02,  2.35000000e+01],
        [-3.59656744e-02, -9.72381248e-01, -4.30672213e-01, ...,
          3.20000000e+02,  1.69500000e+02,  1.14000000e+01],
        ...,
        [-1.27616134e+00,  9.94649575e-01,  2.94206513e-01, ...,
          1.50696336e+02,  0.00000000e+00,  1.38443283e+01],
        [-3.59656744e-02,  2.43973554e-01, -7.54892340e-01, ...,
          6.57176183e+02,  0.00000000e+00,  2.31019546e+01],
        [ 1.20423000e+00,  1.02840321e+00,  3.76944204e-01, ...,
          3.19232139e+02,  0.00000000e+00,  3.28622917e+01]],
       shape=(1668, 11)),
 array(['Ativa', 'Ativa', 'Ativa', ..., 'Sedentária', 'Sedentária',
        'Sedentária'], shape=(1668,), dtype=object))

### 8. Dividindo a Base de Dados entre treino e teste

In [162]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3)

### 9. Treinando os Modelos

#### 9.1 Regressão Logística

In [163]:
modelLogistic = LogisticRegressionCV(solver="liblinear")
modelLogistic.fit(x_train, y_train)
y_pred = modelLogistic.predict(x_test)

In [164]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       Ativa       0.73      0.73      0.73       267
  Sedentária       0.69      0.70      0.70       234

    accuracy                           0.71       501
   macro avg       0.71      0.71      0.71       501
weighted avg       0.71      0.71      0.71       501



#### 9.2 SVM (Support Vector Machines)

In [165]:
modelSVC = SVC()
modelSVC.fit(x_train, y_train)
y_pred = modelSVC.predict(x_test)

In [166]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       Ativa       0.74      0.90      0.81       267
  Sedentária       0.85      0.65      0.73       234

    accuracy                           0.78       501
   macro avg       0.80      0.77      0.77       501
weighted avg       0.79      0.78      0.78       501



#### 9.3 Random Forest Classifier

In [167]:
modelRFC = RandomForestClassifier()
modelRFC.fit(x_train, y_train)
y_pred = modelRFC.predict(x_test)

In [168]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       Ativa       1.00      0.99      1.00       267
  Sedentária       0.99      1.00      1.00       234

    accuracy                           1.00       501
   macro avg       1.00      1.00      1.00       501
weighted avg       1.00      1.00      1.00       501



#### 9.4 Gradient Boosting Classifier

In [169]:
modelGBC = GradientBoostingClassifier()
modelGBC.fit(x_train, y_train)
y_pred = modelGBC.predict(x_test)

In [170]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       Ativa       1.00      0.99      0.99       267
  Sedentária       0.99      1.00      0.99       234

    accuracy                           0.99       501
   macro avg       0.99      0.99      0.99       501
weighted avg       0.99      0.99      0.99       501



#### 9.5 KNeighborsClassifier

In [171]:
modelKNN = KNeighborsClassifier()
modelKNN.fit(x_train, y_train)
y_pred = modelKNN.predict(x_test)

In [172]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       Ativa       0.99      0.83      0.90       267
  Sedentária       0.83      0.99      0.91       234

    accuracy                           0.90       501
   macro avg       0.91      0.91      0.90       501
weighted avg       0.92      0.90      0.90       501



### 10. Importando o melhor modelo

In [ ]:
import joblib 

joblib.dump(modelKNN, "modelKNN.pkl")
joblib.dump(preprocessor, "preprocessor.pkl")